<a href="https://colab.research.google.com/github/HayaLababidi/GP_ArabicTextEmotionRecognition/blob/master/GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Download all files in gdrive folder
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
#local_download_path = os.path.expanduser('~/data')
#try:
#  os.makedirs(local_download_path)
#except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1j0P7oeHOE8eGhzgsaZFkB6tHTrkr5h95' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = f['title']
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())


In [0]:
!pip install gensim
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install nltk


In [0]:
import requests

url = 'https://ia902903.us.archive.org/26/items/full_grams_cbow_300_twitter/full_grams_sg_100_twitter.zip'
r = requests.get(url, allow_redirects=True)
open('full_grams_sg_100_twitter.zip', 'wb').write(r.content)


1132025304

In [0]:
!unzip full_grams_sg_100_twitter.zip

Archive:  full_grams_sg_100_twitter.zip
  inflating: full_grams_sg_100_twitter.mdl  
  inflating: full_grams_sg_100_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_sg_100_twitter.mdl.wv.vectors.npy  


In [0]:
import gensim
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from nltk.stem.isri import ISRIStemmer


_dropout_rate = 0.2
_dropout_rate_softmax = 0.5
_number_of_inputs = 140 #max number of words /characters per doc(tweet)
_vector_size = 100 #vector for each word
_batch_size = 100
_kernal_size= 5 #An integer or tuple/list of a single integer
_pool_size = 3
_noise_shape = (_batch_size,1,_number_of_inputs)
_epochs = 25
_test_size = 0.33 # percentage of test from the dataset
                  
                  
def embed_label(label):
  if label == "anger":
    return 0
  if label == "joy":
    return 1
  if label == "none":
    return 2
  if label == "surprise":
    return 3
  if label == "sadness":
    return 4
  if label == "fear":
    return 5
  if label == "sympathy":
    return 6
  if label == "love":
    return 7
  
  
  
  
#text = input("enter arabic text")

#مسح التشكيل و علامات الترقيم و الحروف المتكررة---------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("إ", "ا", text)
    text = re.sub("أ", "ا", text)
    text = re.sub("آ", "ا", text)
    text = re.sub("ا", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#----------------------------------------------------

#-------------tokenization and stop word removal
def tokens_remove_stopwords(text):

    text = text.split()
    result = list()
    ch = 0

    arabic_stop_words = ["من", "فى", "الي", "علي", "عن", "حتي", "مذ", "منذ", "و", "الا", "او", "ام", "ثم", "بل", "لكن",
                         "كل" , "متى" , "يوم"]

    for word in text:
        for stop_word in arabic_stop_words:
            if word == stop_word:
                ch = 1
                break

        if ch != 1:
            result.append(word)

        ch = 0

    return result
#_______________________________________

#Rooting words
def rooting(text):
    result = list()
    for word in text:
        stemmer = ISRIStemmer()
        result.append(stemmer.stem(word))
    return result

#remove english and empty strings
def remove_english(tokens):
    filtered_tokens = list()
    for word in tokens:
        if (not re.match(r'[a-zA-Z]+', word, re.I)) and word != '':
            filtered_tokens.append(word)
    return filtered_tokens


def preprocess1(text):
    text = str(text)
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_repeating_char(text)
    tokens = re.split(" ", text)
    tokens = remove_english(tokens)
    return tokens

def preprocess2(text):
    text = str(text)
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_repeating_char(text)
    text = tokens_remove_stopwords(text)
    text = remove_english(text)
    text = rooting(text)
    return text

'''
text = normalize_arabic(text)
text = remove_diacritics(text)
text = remove_punctuations(text)
text = remove_repeating_char(text)


print(text)
'''
def embed_doc(text,t_model):
    preprocessed_text = preprocess1(text)
    #print(preprocessed_text)
    
    embedded_vectors = np.zeros(shape=(_number_of_inputs,_vector_size))#np array of arrays (array of 100/300 float number per word)
    for i in range(len(preprocessed_text)):
        try:
            embedded_vectors[i] = t_model.wv[preprocessed_text[i]]
        except:
            try:
                preprocessed_text[i] = rooting([preprocessed_text[i]])[0]
                embedded_vectors[i] = t_model.wv[preprocessed_text[i]]
            except:
                print(preprocessed_text[i] + "word cant be embedded") #currently emojis can't be embedded and for any extreme case (skip wrongly written words)
        #print(tok)
    return embedded_vectors
  



def read_dataset():
    t_model = gensim.models.Word2Vec.load('full_grams_sg_100_twitter.mdl')
    
    data_df = pd.read_csv("Emotional-Tone-Dataset.csv", encoding="windows-1256")
    X = data_df[['tweet']].values
    Y = data_df[['label']].values
    
    eY = np.zeros(shape=(len(Y)))
    for i in range(len(Y)):
        eY[i]= embed_label(Y[i])
    
    X_train, X_test, y_train, y_test = train_test_split(X,eY, test_size = _test_size, random_state=42)
    
    eX_train = np.zeros(shape=(len(X_train),_number_of_inputs,_vector_size))#number of tweets*max number of words per tweet*vector size per word
    eX_test = np.zeros(shape=(len(X_test),_number_of_inputs,_vector_size))
    
    for i in range(len(X_train)):
        eX_train[i]= embed_doc(X_train[i],t_model)

    for i in range(len(X_test)):
        eX_test[i]= embed_doc(X_test[i],t_model)
        
    return eX_train, eX_test, y_train, y_test

                  
X_train, X_test, y_train, y_test = read_dataset()
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)


ميداليهذهبيهword cant be embedded
1928word cant be embedded
راءيلword cant be embedded
وماساوياتيword cant be embedded
40word cant be embedded
ماتنسيناشword cant be embedded
0word cant be embedded
فقيرايرتجفword cant be embedded
يءنword cant be embedded
2016word cant be embedded
محرمالword cant be embedded
4word cant be embedded
بءسword cant be embedded
25word cant be embedded
84word cant be embedded
مباركتيشword cant be embedded
سيخلدالتاريخword cant be embedded
شطءword cant be embedded
ر58word cant be embedded
ءجرword cant be embedded
تءذword cant be embedded
3alahwaword cant be embedded
3word cant be embedded
8word cant be embedded
05b5word cant be embedded
4word cant be embedded
داخرجword cant be embedded
ءلمword cant be embedded
شعءرword cant be embedded
خيلالword cant be embedded
ماتسبينيشword cant be embedded
6word cant be embedded
خءفword cant be embedded
ذءبفword cant be embedded
هضعword cant be embedded
13word cant be embedded
43word cant be embedded
2016word cant be embedded

In [0]:
import numpy
a = numpy.zeros(shape=(5,100),type=float)

In [0]:
import keras
import random
from keras.models import Sequential
import csv
import numpy as np
#import read_data
_dropout_rate = 0.2
_dropout_rate_softmax = 0.5
_number_of_inputs = 140 #max number of words /characters per doc(tweet)
_vector_size = 100 #vector for each word
_batch_size = 10
_kernal_size= 5 #An integer or tuple/list of a single integer
_pool_size = 3
_noise_shape = (_batch_size,1,_number_of_inputs)
_epochs = 100
_test_size = 0.33 # percentage of test from the dataset

'''
#X list of list of arrays
X_train, X_test, y_train, y_test = read_data.read_dataset()
'''

'''
try:
    with open("Embedded_trainX.csv",'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerows(X_train)
    with open("Embedded_testX.csv",'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerows(X_test)
    with open("Embedded_trainY.csv",'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerows(y_train)
    with open("Embedded_testY.csv",'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerows(y_test)
except:
    print("error saving")

X_train = list(csv.reader(open("Embedded_trainX.csv")))
y_train = list(csv.reader(open("Embedded_trainY.csv")))
X_test = list(csv.reader(open("Embedded_testX.csv")))
y_test = list(csv.reader(open("Embedded_testY.csv")))
'''

X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print("done input embedding")
model = Sequential()

#input
#model.add(keras.layers.Input(shape=(_number_of_inputs,_vector_size)))

#Dropout
model.add(keras.layers.Dropout(rate=_dropout_rate,input_shape=(_number_of_inputs,_vector_size)))#,noise_shape,random.randint(0,number_of_inputs)))

#Convolution
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

#Convolution
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

#Convolution
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

#Convolution
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

#Convolution
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))


#Dropout
model.add(keras.layers.Dropout(_dropout_rate_softmax))#,noise_shape,random.randint(0,number_of_inputs)))

#output
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(8, activation="softmax"))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# Train the model
model.fit(
    X_train,
    y_train,
    batch_size=_batch_size,
    epochs=_epochs,
    validation_data=(X_test, y_test),
    shuffle=True
)

(6743, 140, 100)
(6743,)
(3322, 140, 100)
(3322,)
done input embedding
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_31 (Dropout)         (None, 140, 100)          0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 140, 100)          50100     
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 140, 100)          50100     
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 47, 100)           0         
_________________________________________________________________
conv1d_93 (Conv1D)           (None, 47, 100)           50100     
_________________________________________________________________
conv1d_94 (Conv1D)           (None, 47, 100)           50100     
_________________________________________________________________
max_p